##### 문제 1-1 : 기본 체인 만들기 - AI 요리사

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    # model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001A64EC46DB0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001A64F142750> root_client=<openai.OpenAI object at 0x000001A64EBA44D0> root_async_client=<openai.AsyncOpenAI object at 0x000001A64ED35340> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    당신은 AI 요리사입니다. 
    {input}를 사용해서 만들 수 있는 요리를 조리법과 함께 추천해주세요. 
    """)
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    당신은 AI 요리사입니다. \n    {input}를 사용해서 만들 수 있는 요리를 조리법과 함께 추천해주세요. \n    ')

In [12]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [16]:
# chain 호출
try:
    result = chain.invoke({"계란, 밥, 김치"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
{'계란, 밥, 김치'}를 사용해서 만들 수 있는 요리로는 **김치 계란밥**이 있습니다.

### **김치 계란밥**

**재료**: 

*   밥 1공기
*   김치 100g (잘게 썬 것)
*   계란 1개
*   참기름 1티스푼
*   소금 1/2 티스푼
*   후추 1/4 티스푼

**조리법**:

1.  밥을 준비합니다. 밥은 한 공기 정도 준비해 주세요.
2.  김치를 준비합니다. 김치를 잘게 썰어주세요. 
3.  팬에 참기름을 두르고 썰어 놓은 김치를 넣고 볶습니다. 김치가 약간 탄 느낌이 나면 소금과 후추로 간을 해줍니다.
4.  계란을 풀어줍니다. 계란을 잘 풀어서 준비해 주세요.
5.  김치 위에 계란물을 붓고 스크램블 에그처럼 만들어줍니다. 계란이 거의 익으면 불을 끄세요.
6.  밥에 만들어 놓은 김치 계란 혼합물을 넣고 잘 섞어줍니다.

**팁**:

*   간을 조절하고 싶으면 소금과 후추를 추가적으로 넣어주세요.
*   매운맛을 원하면 더 매운 김치를 사용하거나, 청양고추를 추가할 수 있습니다.

이 요리는 간단하지만 맛이 좋습니다. 계란과 김치가 밥과 잘 어울려서 아주 만족스러운 한 끼 식사가 됩니다.


##### 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템 

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 1편 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 한국 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 3줄 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template(
    """
    {movie} 추전한 영화의 제목과 그 영화의 정보(감독, 등장인물)을 설명해주세요. 
    줄을 바꾸어서 영화의 줄거리를 3문장을 세줄로 요약해 주세요.
    """)

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [29]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

**영화 제목:** 올드보이(2003)

**영화 정보:**
- **감독:** 박찬욱
- **출연:** 최민식, 유지태, 강혜정

**영화 줄거리 요약:**

주인공 오대수(최민식)는 자신에게 일어난 모든 일에 대해 복수를 하기 위해 노력합니다. 
그는 자신의 과거와 현재를 되돌아보며, 자신을 가둔 사람의 정체를 찾기 시작합니다.
오대수는 복수를 통해 과거의 트라우마를 극복하려 하지만, 그 과정에서 예상치 못한 진실을 마주하게 됩니다.


In [31]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    """
    당신은 영화 전문가입니다. 
    {genre} 에 맞는 영화를 영화의 제목과 영화의 정보(감독, 등장인물)을 설명하고,
    줄을 바꾸어 영화의 줄거리를 3문장으로 요약해주세요.
    """
)
human_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    human_message
])

# 메시지 생성
messages = chat_prompt.format_messages(genre="액션", 
                                       question="영화를 추천해주세요.")

# LLM 호출
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
response = llm.invoke(messages)

# 결과 출력
print(response.content)

**영화 추천**

*   **영화 제목:** 존 윅
*   **영화 정보:** 
    *   감독: 채드 스태헬스키
    *   등장인물: 키아누 리브스, 마이클 비스프, 알피 앨런, 윌 새미스, 브리짓 무나한

*   **영화 줄거리:** 
    존 윅은 은퇴한 암살자입니다. 
    그러나 그의 은퇴는 오래가지 못하는데, 고인이 된 아내가 남긴 마지막 선물인 '68년형 머스탱'을 훔친 이오간이라는 소년과 그의 패거리들이 존 윅의 집에 찾아오면서부터입니다. 
    존 윅은 이오간과 그의 패거리들을 상대로 싸움을 시작하고, 이 싸움은 점점 더 큰 조직과 연루되면서 그의 과거가 다시 열리게 됩니다.


##### 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 